# Inconsistency

Three claims from the developers. They can't all be true.

1. As Halicarn, you have the option of reclaiming Babylon's final card if they sac it for gold. (https://www.boardgamegeek.com/thread/1764167/complex-timing-question-babylon-halicarnassus-oh-m)
- As Halicarn's neighbor, if you build a vineyard and they reclaim a resource this turn you get the gold from them building "this turn". (https://boardgamegeek.com/thread/1172739/article/15787395#15787395)
- As Babylon, when playing the last two cards in an age, if you gain resources from your turn 6 play you may spend them on your turn 7 play. https://boardgamegeek.com/thread/580321/article/5739388#5739388


### If all three were true at once

1. Babylon plays second card -before- Halicarn reclaims.
2. Halicarn reclaims -before- Vineyard generate gold.
3. Vineyard generates gold -before- Babylon plays second card.

### Keeping (1) & (2) means:
   - babylon active -> they choose a second play
   - execute all chosen plays
       - track vineyards and bazaars to play later
   - discard remaining cards
   - active halicarn -> reclaim
   - vineyards and bazaars generate income
  
Babylon's double-play ability is weaker.

Babylon has two cards remaining (Arena, Academy). They can no longer play Arena and then use the gold to build the academy, they have to choose one or the other.

Babylon builds their wonder with their sixth card, unlocking doubleplay. They can't play their seventh card this age and must discard.

### Keeping (2) & (3) means:
   - execute all chosen plays
       - track vineyards and bazaars to play later
   - if hands have 1 card remaining
       - babylon active -> they hold their last card
       - other cards are discarded
   - halicarn active -> reclaim
   - vineyards and bazaars generate income
   - babylon active -> play card
   
Halicarn has less card selection when Babylon sacs.

If Babylon cannot build either of their last two cards and is forced to sac, Halicarn cannot reclaim that card.

If Babylon sacs a red card for points because it won't win battles, Halicarn cannot reclaim that card.
       
### Keeping (1) & (3) means:
   - execute all chosen plays
       - execute gold cards after brown+grey
   - if hands have 1 card remaining
       - babylon active -> play card
       - other cards are discarded
   - halicarn active -> reclaim
  
Halicarn's neighbors get less gold from Age II commerce.

Building a (vineyard/bazaar) while next to Halicorn, you miss out on (1/2) gold if they reclaim a (brown/grey) card this turn.

This feels the best of the three options. State is simplified, and gold loss is less impactful than card selection.

------


## State and Decisions

Templates for - each card, each civ


(~150 bytes per gamestate)

2 bits (current age)

7 Cities (13 bytes each)

    74 bits, 1 for each unique buildable card
 
    8 bits to track coins
    
    5 bits to track military points won (0-18)
    3 bits to track military defeats (0-6)
    
    3 bits to track which civ
    4 bits to track wonder layers
    1 bit to track if per age action is availible
    
7 "hands" (7 bytes each)

    up to 7 cards, each a 1byte index

1 "discard"

    74 bits, 1 for each unique card


---------

## Decision making

choose: 
- play, build, or sac
- a card from hand

if needed
- choose gold distribution (trade goods)
- choose card to reclaim

your turn
- build: Brick
- sac: Brick
- build: Baths (passing 2 coins left)
- build: Baths (passing 1 coin right)
- sac: Baths
- wonder: Brick (passing 3 coins right)
- wonder: Bath (passing 3 coins right)


### Edge Cases

Olympia FreeBuild
- build: palace for free (using Olympia power & lose flag)
- build: palace (passing 2 left, 4 right)
- build: palace (passing 0 left, 6 right)
- sac: palace
- ...

### Game Flow

Starting
- Card Templates are defined (cards, stats, occurrence)
- Wonder Templates are defined
- Wonders are assigned

Game Loop
- if hands are empty
    - if end of age I, II, or III
        - assign military victories and defeats
    - if end of age III
        - players assign flex guild and flex science to maximize points
        - score point totals
        - end game
   - increment age
   - deal new hands
- if hands have 2+ cards
   - rotate hands
   - players look at cards; each player independently chooses a play 
       - eg (build/sac/wonder) (CARD) passing (2) gold left (3) gold right
   - execute all chosen plays
       - execute gold cards after brown+grey
   - if hands have 1 card remaining
       - if babylon active -> choose and make play
       - other cards are discarded
   - if halicarn active -> reclaim


In [ ]:
class Wonder:
    pass

"""
Alexandria
    +glass
    
    2stone -> 3vp
    2ore -> choice of raw resources
    2glass -> 7vp
    
    2brick -> choice of raw resources
    2wood -> choice of crafts
    3stone -> 7vp
    
Great Pyramid
    +stone
    
    2stone -> 3vp
    3wood -> 5vp
    4stone -> 7vp
    
    2wood -> 3vp
    3stone -> 5vp
    3brick -> 5vp
    4stone 1paper -> 7vp

Olympia
    +wood
    
    2wood -> 3vp
    2stone -> Once per age, construct any for free
    2ore -> 7vp
    
    2wood -> two sided trade post
    2stone -> 5vp
    2ore 1cloth -> copy neighboring guild

Artemis
    +paper
    
    2 stone -> 3vp
    2 wood -> 9gold
    2 paper -> 7vp
    
    2 stone -> 2vp 4gold
    2 wood -> 3vp 4gold
    glass paper cloth -> 5vp 4gold

Rhodos
    +ore
    
    2 wood -> 3vp
    3 brick -> 2 military
    4 ore -> 7vp
    
    3 stone -> 1 military 3vp 3gold
    4 ore -> 1 military 4vp 4gold

Halicarnassos
    +cloth
    
    2 brick -> 3vp
    3 ore -> reclaim (wait until discards, pick and build from discard)
    2 cloth -> 7vp
    
    2 ore -> reclaim
    3 brick -> reclaim
    glass paper cloth -> reclaim

Babylon
    +brick
    
    2 brick -> 3vp
    3 wood -> Choice of Science
    4 brick -> 7vp
    
    1 cloth -> 3vp
    2 wood 1 glass -> Play Last Card of Each Age
    3 brick 1 paper -> Choice of Science

"""

In [ ]:
class Card:
    # grey
    Cloth = 0
    Glass = 1
    Paper = 2
    
    # brown
    Trees = 3
    Metal = 4
    Brick = 5
    Stone = 6
    StoneTrees = 7
    BrickMetal = 8
    StoneBrick = 9
    TreesMetal = 10
    TreesBrick = 11
    BrickMetal = 12
    TreesTwo = 13
    MetalTwo = 14
    BrickTwo = 15
    StoneTwo = 16
    
    # gold
    TradeEast = 17
    TradeWest = 18
    Forum = 19
    Haven = 20
    Marketplace = 21 
    Caravansery = 22
    Lighthouse = 23
    Tavern = 24
    Vineyard = 25
    Bazaar = 26
    Arena = 27
    ChamberOfCommerce = 28 
    
    # blue
    Altar = 29
    Temple = 30
    Pantheon = 31
    Theatre = 32
    Statue = 33
    Gardens = 34
    Baths = 35
    Aqueduct = 36 
    Pawnshop = 37
    Courthouse = 38
    Senate = 39
    TownHall = 40
    Palace = 41
    
    #red
    Stockade = 42
    Barracks = 43
    GuardTower = 44
    Stables = 45
    ArcheryRange = 46 
    Walls = 47
    TrainingGround = 48
    SiegeWorkshop = 49
    Fortification = 50
    Arsenal = 51
    Circus = 52
    
    #green
    Apothecary = 53
    Workshop = 54
    Scriptorium = 55 
    Library = 56
    Laboratory = 57
    Dispensary = 58
    School = 59
    University = 60 
    Observatory = 61
    Lodge = 62
    Study = 63
    Academy = 64
    
    #purple
    BrownGuild = 65 
    GreyGuild = 66
    GoldGuild = 67
    GreenGuild = 68
    RedGuild = 69
    BlueGuild = 70
    StrategyGuild = 71
    ScientistGuild = 72
    ShipownerGuild = 73
    BuilderGuild = 74
    
    
    
    
    

class CardTemplate:
    pass


In [ ]:
import numpy as np

class City:
    def __init__(self):
        self.built = np.zeros(74).astype(np.bool)
        self.gold = 3
        
        self.stage1 = False
        self.stage2 = False
        self.stage3 = False

In [21]:
np.zeros(7).astype(np.bool)

array([False, False, False, False, False, False, False])

In [16]:
import itertools
import numpy as np

Trees = 0
Metal = 1
Brick = 2
Stone = 3
Cloth = 4
Glass = 5
Paper = 6

class ResourceSet:
    def __init__(self):
        self.possible = [np.zeros(7),]
        
    def __add__(self, other):
        new = []
        for a,b in itertools.product(self.possible, other.possible):
            if a+b not in new:
                new.append(a+b)

        
test = ResourceSet()

test.possible[0][Metal] = 1

test.possible


[array([0., 1., 0., 0., 0., 0., 0.])]

In [ ]:
class Color:
    Brown = 0
    Grey = 1
    Gold = 2
    Blue = 3
    Red = 4
    Green = 5
    Purple = 6
    
class Card:
    def __init___(self, name, color):
        self.name = name
        self.color = color

class Land(Card):
    def __init__(self, resource, color=Color.Brown, amount=1):
        super().__init__(color)
        self.resources = ResourceSet()
        self.resources[0][resource] = amount
        
    def __and__(self, other):
        
        

        
Age1 = [
    Land(Wood), Land(Wood),
    Land(Wood)
]